In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN_Wmultitask_64_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 5    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
train_data.shape

(691, 1024, 77)

In [10]:
pos_weight.shape

(19,)

In [11]:
pos_weight

array([  0.16329966,  34.21019108,  34.43589744,  37.12413793,
        36.10067114,  26.70927318,  27.27621483,  39.94814815,
        42.01945525,  46.65517241,  51.89952153,  48.57847534,
        51.1509434 ,  43.40160643,  44.68595041,  33.7672956 ,
        10.65015806,  36.60544218,   2.20649652])

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/checkpoint' 

In [14]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [15]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet = True,
                                 trainable_conv = True,
                                 trainable_last = True,
                                 variables_weights = variables_weights)

In [16]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [17]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.072 vs. 0.078
F1 = 0.111 vs. 0.119
Accuracy = 0.016 vs. 0.056
CHANGE DETECTION
AUC = 0.317 vs. 0.264
F1 = 0.500 vs. 0.420
Precision = 0.358 vs. 0.266
Recall = 0.830 vs. 0.998
Validation
ACTIVITY RECOGNITION
AUC = 0.072 vs. 0.072
F1 = 0.111 vs. 0.099
Accuracy = 0.008 vs. 0.056
CHANGE DETECTION
AUC = 0.224 vs. 0.186
F1 = 0.421 vs. 0.313
Precision = 0.280 vs. 0.186
Recall = 0.852 vs. 1.000
Minibatch Loss= 1.251682
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.539 vs. 0.081
F1 = 0.571 vs. 0.125
Accuracy = 0.544 vs. 0.056
CHANGE DETECTION
AUC = 0.723 vs. 0.320
F1 = 0.716 vs. 0.475
Precision = 0.671 vs. 0.312
Recall = 0.767 vs. 0.992
Validation
ACTIVITY RECOGNITION
AUC = 0.335 vs. 0.070
F1 = 0.417 vs. 0.102
Accuracy = 0.488 vs. 0.056
CHANGE DETECTION
AUC = 0.489 vs. 0.183
F1 = 0.580 vs. 0.313
Precision = 0.469 vs. 0.185
Recall = 0.761 vs. 1.000
Minibatch Loss= 0.407080
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.720 vs. 0.078


In [18]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/DWN_Wmultitask_64_opp/checkpoint-20000'

In [19]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../../model/DWN_Wmultitask_64_opp/checkpoint-20000')


INFO:tensorflow:Restoring parameters from ../../model/DWN_Wmultitask_64_opp/checkpoint-20000


In [20]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.602 vs. 0.070
F1 = 0.620 vs. 0.099
Accuracy = 0.914 vs. 0.056
CHANGE DETECTION
AUC = 0.584 vs. 0.177
F1 = 0.596 vs. 0.311
Precision = 0.555 vs. 0.184
Recall = 0.644 vs. 1.000


In [21]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [22]:
out.shape

(119, 16, 19)

In [23]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [23]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [21]:
1

1